## Data correction procedure for TOF-SANS with polarization analysis

- Note on notation:
> - subscripts = discretization variable from measurement
> - superscripts = Names
> - () = continuous function of....

| Value | Meaning | shape | How acquired? | 
|:--------:|:--------:|:--------:|:--------:|
|  $I^{++}_{t,\lambda}$   |  uncorrected sample data of ++ channel |  data array with columns for $I$, $\lambda$, $t$   |  neutron data |
|  $I^{+-}_{t,\lambda}$   |  uncorrected sample data of +- channel |  data array with columns for $I$, $\lambda$, $t$   | neutron data    |
|  $I^{-+}_{t,\lambda}$   |  uncorrected sample data of -+ channel  |  data array with columns for $I$, $\lambda$, $t$   | neutron data   |
|  $I^{--}_{t,\lambda}$   |  uncorrected sample data of -- channel |  data array with columns for $I$, $\lambda$, $t$    | neutron data   |
|  $S^{++}_{t,\lambda}$   |  corrected sample data of ++ channel  |  data array with columns for $I$, $\lambda$, $t$   | by pol-correction  |
|  $S^{+-}_{t,\lambda}$   |  corrected sample data of +- channel  |  data array with columns for $I$, $\lambda$, $t$   | by pol-correction   |
|  $S^{-+}_{t,\lambda}$   |  corrected sample data of -+ channel  |  data array with columns for $I$, $\lambda$, $t$   | by pol-correction   |
|  $S^{--}_{t,\lambda}$   |  corrected sample data of -- channel  |  data array with columns for $I$, $\lambda$, $t$   | by pol-correction    |
|  $p^{cell}$   |  pressure of <sup>3</sup>He-polarizer/analyzer  |  scalar, 1-dim   | from metadata  |
|  $l^{cell}$   |  length of <sup>3</sup>He-polarizer/analyzer   |  scalar, 1-dim   | from metadata   |
|  $T^{g, cell}(\lambda)$   |  transmission of empty glass of polarizer/analyzer   |  scalar, 1-dim   | from metadata  |
|  $O^{cell}(\lambda)=O^{0, cell}\cdot\lambda$   |  wavelength-dependent opacity of <sup>3</sup>He-polarizer/analyzer  |  continuous function of $\lambda$   | $O^{0, cell}$ fit to data |
|  $O^{0, cell}$   |  wavelength-independent pre-factor of opacity of <sup>3</sup>He-polarizer/analyzer   |  scalar, 1-dim   | fit to data  |
|  $P^{^3He, cell}(t) = C^{cell}e^{-t/T^{1, cell}}$   |  time-dependent nuclear polarization of <sup>3</sup>He-polarizer/analyzer  |  continuous function of $t$   | $C^{cell}, T^{1, cell}$ fit to data   |
|  $C^{cell}$   |  Pre-factor of nuclear polarization exponential decay of <sup>3</sup>He-polarizer/analyzer  |  scalar, 1-dim   | fit to data | $P_{He0}$   |
|  $T^{1, cell}$   |  Decay constant of nuclear polarization exponential decay of <sup>3</sup>He-polarizer/analyzer  |  scalar, 1-dim   | fit to data |
|  $I^{DB,unpol-in,cell}_{\lambda, t}$   |  Intensity of direct-beam of an unpolarized-incoming beam to the polarized <sup>3</sup>He-polarizer/analyzer  |  data array with columns for $I$, $\lambda$, $t$   | neutron data  |
|  $I^{DB,depol,cell}_{\lambda}$   |  Intensity of direct-beam of an unpolarized-incoming beam to the depolarized <sup>3</sup>He-polarizer/analyzer |  data array with columns for $I$, $\lambda$  | neutron data  ||
|  $I^{DB, no-cell}_{\lambda}$   |  Intensity of direct-beam of an unpolarized-incoming beam to detector without passing cells or sample  |  data array with columns for $I$, $\lambda$  | neutron data   |
|  $\hat{T}^{\pm, cell}(t,\lambda)$   |  Transmission-function of a in-coming polarized neutron beam into <sup>3</sup>He-polarizer/analyzer  |  Row 1   | by inserting  $P^{^3He, cell}(t)$,  $O^{cell}(\lambda)$ |



### Overview and general workflow

Different to unpolarized SANS, the spin-dependent cross section $\vec{S}$ after sample-scattering are correlated and cannot be corrected individually [Ref 1, 2, 3]. Instead, a beamline-component dependent correction Matrix $\hat{M}$ has to be applied to all spin-dependent measurements simultaneously to retrieve $\vec{S}$ from the measured intensities $\vec{I}$:

$$
\begin{pmatrix} I^{++}_{t,\lambda} \\ I^{+-}_{t,\lambda}  \\ I^{-+}_{t,\lambda} \\  I^{--}_{t,\lambda}\end{pmatrix} = \hat{M}(t,\lambda) 
\begin{pmatrix} S^{++}_{t,\lambda} \\ S^{+-}_{t,\lambda}  \\ S^{-+}_{t,\lambda} \\  S^{--}_{t,\lambda}\end{pmatrix}
$$

Here, we describe the correction of TOF-SANS data with polarization analysis of the esspolarization package using two time-dependent <sup>3</sup>He-polarized cells. Hence, the polarization-related beamline components in $\hat{M}$ are the time- and spin-dependent transmission matrices of the <sup>3</sup>He-polarizer $\hat{T^p}$ and <sup>3</sup>He-analyzer $\hat{T^a}$: 

$$
\begin{pmatrix} I^{++}_{t,\lambda} \\ I^{+-}_{t,\lambda}  \\ I^{-+}_{t,\lambda} \\  I^{--}_{t,\lambda}\end{pmatrix} = \hat{T}^a(t,\lambda)\cdot\hat{T}^p(t,\lambda) 
\begin{pmatrix} S^{++}_{t,\lambda} \\ S^{+-}_{t,\lambda}  \\ S^{-+}_{t,\lambda} \\  S^{--}_{t,\lambda}\end{pmatrix}
$$



Both $\hat{T^p}$ and $\hat{T^a}$ can be derived from their cell parameters (the transmission of the empty cell $T_g$,  and the cell opacity $O$), and the time-dependent nuclear polarization of the cell $P_{^3He}(t)$.<br>
This general method for time-dependent <sup>3</sup>He-cells can be adapted for other use-cases such as using supermirrors or in-situ polarized ^3He-cells, see "Note" at the end of the page. 

First, the created test-data needs to be readout and grouped by purpose, e.g., if it describes a sample-run, a direct-beam-run, or a background measurement without cells and sample. All data will be reduced using the normalization term described in https://scipp.github.io/ess/techniques/sans/sans-beam-center-finder.html. Second, for the calculation of time-dependent $P_{^3He}(t)$, the ROI's of the direct-beam region and background region have to be defined (USING BEAMCENTER FINDER OPTION OF SANS2D?). Third, we will describe the data correction workflow with a calculation of $O$, $P_{^3He}(t)$, and $T^{\pm}$ of the polarizer and analyzer, and the application on the sample-data.

To facilitate an in-situ data correction during the experiments, we have implemented two different approaches: (i) a rough data-correction based on a <sup>3</sup>He-cell opacity calculated by its cell parameters and filling pressure ("workflow 1") which can be used in-situ during the experiment, and (ii) a precise calculation based on the calculation of opacity by a direct-beam measurement on the depolarized cell at its end of lifetime (i.e., end of cell-usage for this experiment) ("workflow 2")

### Data correction

#### Calculation of <sup>3</sup>He opacity 
The <sup>3</sup>He opacity is a cell-specific parameter and can vary after each filling process. Hence, for precise data correction, it must be measured for each newly filled cell. It linearly depends on the neutron wavelength as follows:

$$
O^{cell}(\lambda)=O^{0, cell}\cdot\lambda
$$

where the wavelength-independent parameter $O^{0, cell}$ depends on cell properties and filling pressure. In the following, two different approaches are presented to measure $O^{0, cell}$:

$\mathbf{Workflow 1}$:<br>
For an in-situ quick data correction, a measure of the cell pressure $p$ at 20C during filling the cell can be used to calculate $O^{0, cell}$ by following relation (Ref 1):

$$
O^{0, cell}= 0.0733 \cdot p^{cell} \cdot l^{cell}
$$

with a factor 0.0733 arising from the number density of <sup>3</sup>He gas, the path length $l$ of neutron through the cell, and the absorption cross section of 1$\mathring A$ neutrons by <sup>3</sup>He. However, a measurement of the opacity by neutron transmission in a depolarized cell condition yields more precise values and can be performed at the end of the cell-lifetime during the user's neutron experiments, i.e., leading to "workflow 2". 

- Note: $l$ is assumed constant over the area of the <sup>3</sup>He-cell. In case of too large cell volumes, only the fraction with constant $l$/ constant $O$ will be used for the neutron measurements. The same assumption holds for both SANS-cells as well as wide-angle cells
- Note: until now, we have treated $T^{g, cell}$ as constant parameter and ignored its wavelength-dependence, as its variation with wavelength is small enough to not impact the correction procedure. However, to keep it generic and enable to insert a wavelength-dependent $T^{g, cell}(\lambda)$, the dependence on wavelength will be pre-characterized (during fabrication of the cell) and inserted as function into this workflow instead of a scalar.

$\mathbf{Workflow 2}$:<br>
A more precise data correction is based on a direct-beam measurement through the depolarized cell (i.e., $P_{^3He}(t) \rightarrow 0$)(Ref. 2). For such a depolarized cell and an unpolarized incident beam (i.e., $\hat{T}^{depol} = (\hat{T}^++\hat{T}^-)/2$), the neutron transmission becomes:

$$
T^{depol, cell}(\lambda)=T^{g, cell}(\lambda)e^{-O^{0, cell}\cdot\lambda}
$$

This leads to following direct beam intensity $I^{DB,depol,cell}_{\lambda}$ measured on a depolarized cell:

$$
\frac{I^{DB,depol,cell}_{\lambda}}{I^{DB,no-cell}_{\lambda}}=T^{g, cell}(\lambda)e^{-O^{0, cell}\cdot\lambda}
$$

where $I^{DB,no-cell}_{\lambda}$ is the intensity of a direct beam through an open neutron path (without cell and sample) and acts as a background measurement. For a measurement of $I^{DB,depol,cell}_{\lambda}$, the <sup>3</sup>He cell needs to be depolarized at the end of its lifetime and $I^{DB,depol,cell}_{\lambda}$ has to be acquired for each cell (and the other cell and sample out of beam). 

- Note: Workflow 2 should overwrite workflow 1 opacity after the experiment
- Note: Different to the sample data, $I^{DB,depol,cell}$ and $I^{DB,no-cell}$ are not time-dependent
- Note: It is important to fit $I_{depol}/I^{DB, no-cell}_{\lambda}$ instead of $I_{depol}$, such that all the parameters in the fit-functions are scalars.


#### Calculation of time-dependent <sup>3</sup>He-polarization
  
Now $O^{cell}(\lambda)=O^{0, cell}\cdot\lambda$ is known. <br>
The next step for getting the transmission functions $\hat{T}^{\pm}$ is to calculate the nuclear polarization of the <sup>3</sup>He-cell, which decays exponentially with a decay-time $T^{1, cell}$ and is given by:

$$
P^{^3He, cell}(t) = C^{cell}e^{-t/T^{1, cell}}
$$.

From the neutron measurement, this again can be retrieved by measuring the transmission through each cell (i.e., for an unpolarized incoming beam into the polarized cell):

$$
\hat{T}^{unpol-in}(\lambda, t) = T^{g, cell}(\lambda)e^{-O^{0, cell}\cdot\lambda}\cosh({P_{^3He}(t)\cdot O^{0, cell}\cdot\lambda})
$$<br>
$$
\frac{I^{DB,unpol-in,cell}_{\lambda, t}}{I^{DB, no-cell}_{\lambda}} = T^{g, cell}(\lambda)e^{-O^{0, cell}\cdot\lambda}\cosh({P^{^3He, cell}(t)\cdot O^{0, cell}\cdot\lambda})
$$

Fitting this equation to the direct-beam data will yield $C^{cell}$ and $T^{1, cell}$, which then can be used to calculate $P^{^3He, cell}(t)$, which shall be the output of the procedure. 

- Note: Some instruments cannot move the polarizer, and in those cases the incoming beam on the analyzer would be polarized. In such cases, another workflow has to be used.
- Note: The readout of which spin-channel to use for $I^{DB,unpol-in,cell}_{\lambda, t}$ is irrelevant, as they are the same.

#### Spin-dependent transmission values

Now both $O^{cell}(\lambda)=O^{0, cell}\cdot\lambda$ and $P^{^3He, cell}(t)$ are known. <br>
Following Refs (1, 3), the polarization-dependent transmission matrices of a polarized neutron beam through the ^3He cells can be computed by inserting 

$$
\hat{T}^{\pm, cell}(t,\lambda) = T^g\cdot e^{-O^{0, cell}\cdot\lambda\pm O^{0, cell}\cdot\lambda\cdot P^{^3He, cell}(t)}
$$

into:

$$
\hat{T}^{p}=
\begin{pmatrix} 
T^{p,+}(t,\lambda) & 0 & T^{p,-}(t,\lambda) & 0 \\ 0 & T^{p,+}(t,\lambda) & 0 & T^{p,-}(t,\lambda) \\ T^{p,-}(t,\lambda) & 0 & T^{p,+}(t,\lambda) & 0 \\ 0 & T^{p,-}(t,\lambda) & 0 & T^{p,+}(t,\lambda)  
\end{pmatrix}
$$


$$
\hat{T}^{a}=
\begin{pmatrix} 
T^{a,+}(t,\lambda) &  T^{a,-}(t,\lambda) & 0 &0\\ T^{a,-}(t,\lambda) &  T^{a,+}(t,\lambda) & 0 & 0\\ 0&0& T^{a,+}(t,\lambda) &  T^{a,-}(t,\lambda) \\ 0&0& T^{a,-}(t,\lambda) &  T^{a,+}(t,\lambda)
\end{pmatrix}
$$

- Note: same function for both cells, but with different input parameters
- Note: in principle the incoming beam to the polarizer is unpolarized, so $\hat{T}^{unpol-in}(\lambda, t)$ isntead of $\hat{T}^{\pm, cell}(t,\lambda)$ could be used. However, the results will be the same in this case without the need to differ between the cases.

#### Sample data correction

For polarizing <sup>3</sup>He-cells without neutron flippers in the beamline, the correction matrix $\hat{M}$  becomes $\hat{M} = \hat{T}^{a}\hat{T}^{p}$. To correct the measured data I, we need to apply the inverse matrix to the data $(\hat{M})^{-1}$:

$$
\begin{pmatrix} I^{++}_{t,\lambda} \\ I^{+-}_{t,\lambda}  \\ I^{-+}_{t,\lambda} \\  I^{--}_{t,\lambda}\end{pmatrix} = \hat{T}^a(t,\lambda)\cdot\hat{T}^p(t,\lambda) 
\begin{pmatrix} S^{++}_{t,\lambda} \\ S^{+-}_{t,\lambda}  \\ S^{-+}_{t,\lambda} \\  S^{--}_{t,\lambda}\end{pmatrix}
$$

$$
\left(\hat{T}^a(t,\lambda)\cdot\hat{T}^p(t,\lambda)\right)^{-1} \begin{pmatrix} I^{++}_{t,\lambda} \\ I^{+-}_{t,\lambda}  \\ I^{-+}_{t,\lambda} \\  I^{--}_{t,\lambda}\end{pmatrix} = 
\begin{pmatrix} S^{++}_{t,\lambda} \\ S^{+-}_{t,\lambda}  \\ S^{-+}_{t,\lambda} \\  S^{--}_{t,\lambda}\end{pmatrix}
$$

- Note: probably easier to first put functions into the matrix product $T^a$ $\cdot$ $T^p$, then numerically get the inverse of that matrix product.
- Note: we need to perform computation of matrix on channels all together, so we always need all four sets of T+-, a,p together

### Literature: 
1) Wai Tung Lee et al., Polarisation Development at the European Spallation Source
2) Kathryn Krycka et al., Polarization-analyzed small-angle neutron scattering. I. Polarized data reduction using Pol-Corr
3) A. Wildes, The polarizer-analyzer correction problem in neutron polarization analysis experiments
4) T. Gentile et al., Optically polarized <sup>3</sup>He

Note 1:<br>
This workflow can be applied on polarization analyzed sample-data using two time-dependent ex-situ polarized <sup>3</sup>He-based spin filters as both polarizer and analyzer. To be used for the instruments SKADI and DREAM, following adaptations have to be made:

1: DREAM-SANS:<br>
operated with a movable in-situ SEOP <sup>3</sup>He-polarizer and a movable ex-situ SEOP small-angle <sup>3</sup>He-analyzer (nm-SANS-option):<br>
--> the polarizer is continuously pumped and considered as time-independent. Values like the opacity, nuclear polarization, and transmission will be pre-characterized and inserted to the workflow as constant parameters.

2: DREAM-Diffraction:<br>
operated with a movable in-situ SEOP <sup>3</sup>He-polarizer and a NOT-movable ex-situ MEOP wide-angle <sup>3</sup>He-analyzer <br>
--> for the polarizer, the same treatment as stated in 1. will be applied. For the analyzer a new workflow considering that the analyzer cannot be moved out of the beam has to be written.

3: SKADI:<br>
operated with a movable supermirror V-cavity polarizer, a spin flipper, and a movable ex-situ SEOP/MEOP <sup>3</sup>He-analyzer:<br>
--> The supeprmirror polarizer (time-independent and wavelength-dependent) and spin flipper transmission values can be pre-characterized and inserted to the workflow as constant parameters. For the analyzer, the current workflow can be adapted.



### Open questions regarding DREAM:

- Should the incident beam monitor be moved behind the polarizer?
- --> we would need a monitor after polarizer if analyzer cannot be moved and polarizer also needs to be characterized
- --> current decision: moving monitor to after polarizer (but on another stage, so moves of polarizer do not move monitor
- --> The "unpolarized SANS" data correction then will be taken on the monitor AFTER polarizer. But as it is an in-situ polarizer with constant polarization, this just leads to a constant offset which will be corrected for using this workflow. If sudden fluctuations in intensity are observed, the polarizer has to be moved out to characterize the neutron beam from the target.
- How to treat an in-situ polarizer?
- --> instead of exponential decay use a step-function, something like: "if C changes, then shift P" - basically constant in time, but with adaptable C (piecewise constant function)
- How to adapt workflow for the non-movable wide-angle analyzer?
- --> needs to be changed as following: instead of moving analyzer out during measuring P, measure with different monitor for P, and just move out polarizer for measuring analyzer
- How to treat the "SANS-window" of the high-angle analyzer?
- --> characterizing the wide-angle would be done outisde user experiments, just measured sometimes in between by beamline scientists - here then parameters are treated constant for both parts (big part and SANS-window-part)